# Pipeline ETL - Projeto de integração de dados - Grupo 06

## Imports e setup

In [1]:
!pip install pandas ipython

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 18.0 MB/s eta 0:00:00


In [2]:
import pandas as pd
from IPython.display import HTML
from glob import glob

## Informação dos dados

De [OpenDataSUS Sinan/Dengue](https://opendatasus.saude.gov.br/dataset/arboviroses-dengue):


> O Sistema de Informação de Agravos de Notificação (Sinan) tem como objetivo coletar, transmitir e disseminar dados gerados rotineiramente pela vigilância epidemiológica das três esferas de governo, por meio de uma rede informatizada, para apoiar o processo de investigação e dar subsídios à análise das informações das doenças e dos agravos de notificação compulsória. Atualmente, o sistema possui duas versões vigentes, Sinan Online e Sinan Net.

> O Sinan Online visa à inserção e disseminação dos dados de notificação e investigação de dengue e de febre de chikungunya, enquanto que o Sinan Net é alimentado pela notificação e investigação da grande maioria dos agravos e doenças, que constam na Lista Nacional de Notificação Compulsória de Doenças, Agravos e Eventos de Saúde Pública, do Anexo 1 do Anexo V da Portaria de Consolidação nº 4, de 28 de setembro de 2017, que consolida as normas sobre os sistemas e os subsistemas do Sistema Único de Saúde, mas é facultado a estados e municípios incluir outros problemas de saúde importantes para o seu contexto local.

> Destaca-se que a dengue é doença de notificação compulsória, ou seja, todo caso suspeito e/ou confirmado deve ser obrigatoriamente notificado ao Serviço de Vigilância Epidemiológica da Secretaria Municipal de Saúde (SMS). As notificações de casos suspeitos de dengue devem ser registradas na Ficha de Notificação/Investigação da dengue e chikungunya e inseridas no Sistema de Informação de Agravos de Notificação – Sinan Online. Os óbitos suspeitos pela infecção do vírus dengue (DENV) são de notificação compulsória imediata para todas as esferas de gestão do Sistema Único de Saúde (SUS), a ser realizada em até 24 horas a partir do seu conhecimento, pelo meio de comunicação mais rápido disponível. Posteriormente, os dados devem ser inseridos no Sistema de Informação de Agravos de Notificação (Sinan Online).


Para informações específicas das colunas, consultar o [Dicionário de dados](https://s3.sa-east-1.amazonaws.com/ckan.saude.gov.br/SINAN/Dengue/dic_dados_dengue.pdf).


## Extração dos dados

Baixando os dados. Obtemos os dados de 2020 à 2022 pois o uso de memória é menor que o dos registros mais recentes (e mesmo assim temos por volta de 3G de RAM sendo utilizadas para carregar os dados)

In [3]:
MIN_YEAR = 20
MAX_YEAR = 22

In [4]:
for year in range(MIN_YEAR, MAX_YEAR + 1):
    url = f"https://s3.sa-east-1.amazonaws.com/ckan.saude.gov.br/SINAN/Dengue/csv/DENGBR{year}.csv.zip"
    !wget "$url"
    !unzip -o "DENGBR{year}.csv.zip"
!rm -f *.zip # deleting all zip files

--2025-12-02 20:39:43--  https://s3.sa-east-1.amazonaws.com/ckan.saude.gov.br/SINAN/Dengue/csv/DENGBR20.csv.zip
Resolving s3.sa-east-1.amazonaws.com (s3.sa-east-1.amazonaws.com)... 52.95.163.35, 16.12.1.36, 3.5.232.44, ...
Connecting to s3.sa-east-1.amazonaws.com (s3.sa-east-1.amazonaws.com)|52.95.163.35|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 51373250 (49M) [application/zip]
Saving to: ‘DENGBR20.csv.zip’

DENGBR20.csv.zip    100%[===================>]  48.99M  8.44MB/s    in 5.8s    

2025-12-02 20:39:51 (8.44 MB/s) - ‘DENGBR20.csv.zip’ saved [51373250/51373250]

Archive:  DENGBR20.csv.zip
  inflating: DENGBR20.csv            
--2025-12-02 20:39:59--  https://s3.sa-east-1.amazonaws.com/ckan.saude.gov.br/SINAN/Dengue/csv/DENGBR21.csv.zip
Resolving s3.sa-east-1.amazonaws.com (s3.sa-east-1.amazonaws.com)... 3.5.234.206, 3.5.233.14, 3.5.234.173, ...
Connecting to s3.sa-east-1.amazonaws.com (s3.sa-east-1.amazonaws.com)|3.5.234.206|:443... connected.
HTTP r

Carregando os arquivos individuais e juntando os DataFrames em um só

ATENÇÃO: ALTO USO DE RAM

In [5]:
common_name = "DENGBR{}.csv"

# Start with columns from the first file
sample_path = common_name.format(MAX_YEAR)
master_cols = set(pd.read_csv(sample_path, nrows=0).columns.tolist())

column_tracker = {}

all_dataframes = []

for year in range(MIN_YEAR, MAX_YEAR + 1):
    path = common_name.format(year)

    df = pd.read_csv(path, encoding='utf-8')

    for col in df.columns:
        if col not in column_tracker:
            column_tracker[col] = []
        column_tracker[col].append(year)

    all_dataframes.append(df)

# Convert master_cols to ordered list (optional: sort)
master_cols = sorted(master_cols)

# Normalize columns across all dataframes
for i in range(len(all_dataframes)):
    df = all_dataframes[i]
    for col in master_cols:
        if col not in df.columns:
            df[col] = pd.NA
    all_dataframes[i] = df[master_cols]

# Final concatenation
df = pd.concat(all_dataframes, ignore_index=True)

/tmp/ipython-input-4107040436.py:14: DtypeWarning: Columns (22,44,45,46,54,56,101) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(path, encoding='utf-8')
/tmp/ipython-input-4107040436.py:14: DtypeWarning: Columns (22,44,45,46,54,101) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(path, encoding='utf-8')
/tmp/ipython-input-4107040436.py:14: DtypeWarning: Columns (22,44,45,46,54,101) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(path, encoding='utf-8')
/tmp/ipython-input-4107040436.py:35: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat(all_dataframes, ignore_index=True)


In [6]:
del all_dataframes

## Transformação dos dados

In [7]:
df.info(verbose=True, show_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3899353 entries, 0 to 3899352
Data columns (total 121 columns):
 #    Column      Non-Null Count    Dtype  
---   ------      --------------    -----  
 0    ACIDO_PEPT  3782453 non-null  float64
 1    ALRM_ABDOM  38132 non-null    float64
 2    ALRM_HEMAT  37975 non-null    float64
 3    ALRM_HEPAT  37959 non-null    float64
 4    ALRM_HIPOT  38083 non-null    float64
 5    ALRM_LETAR  38005 non-null    float64
 6    ALRM_LIQ    37971 non-null    float64
 7    ALRM_PLAQ   38167 non-null    float64
 8    ALRM_SANG   38066 non-null    float64
 9    ALRM_VOM    38043 non-null    float64
 10   ANO_NASC    2389636 non-null  float64
 11   ARTRALGIA   3782453 non-null  float64
 12   ARTRITE     3782453 non-null  float64
 13   AUTO_IMUNE  3782453 non-null  float64
 14   CEFALEIA    3782453 non-null  float64
 15   CLASSI_FIN  3895398 non-null  float64
 16   CLINC_CHIK  22748 non-null    float64
 17   COMPLICA    0 non-null        float64
 18   

In [8]:
total_samples = len(df)

### Verificando quantidade de valores únicos

In [9]:
unique_value_counts = df.nunique(dropna=False)

In [10]:
HTML(unique_value_counts.to_frame("n_unique").to_html(max_rows=None))

,n_unique
ACIDO_PEPT,3
ALRM_ABDOM,3
ALRM_HEMAT,3
ALRM_HEPAT,3
ALRM_HIPOT,3
ALRM_LETAR,3
ALRM_LIQ,3
ALRM_PLAQ,3
ALRM_SANG,3
ALRM_VOM,3


Seguindo o dicionário de dados, `ID_AGRAVO` diz respeito ao código internacional de doenças (CID-10). Na base de dados adquirida, apenas um valor possível é encontrado: A90 (Dengue). Como essa coluna é não-informativa, podemos removê-la para economizar espaço


In [11]:
df.drop("ID_AGRAVO", axis=1, inplace=True)

TODO: Outros elementos possuem apenas um valor úncio mas precisa ser investigado se podem ser removidos também

### Verificando conteúdo dos valores únicos

In [12]:
columns_of_interest = unique_value_counts[unique_value_counts <= 30].index.to_list()
for column in columns_of_interest:
    if column in df.columns:
        print("=" * 10, column, "="*10)
        print(df[column].unique())

========== ACIDO_PEPT ==========
[ 2.  1. nan]
========== ALRM_ABDOM ==========
[nan  2.  1.]
========== ALRM_HEMAT ==========
[nan  2.  1.]
========== ALRM_HEPAT ==========
[nan  2.  1.]
========== ALRM_HIPOT ==========
[nan  2.  1.]
========== ALRM_LETAR ==========
[nan  2.  1.]
========== ALRM_LIQ ==========
[nan  2.  1.]
========== ALRM_PLAQ ==========
[nan  2.  1.]
========== ALRM_SANG ==========
[nan  1.  2.]
========== ALRM_VOM ==========
[nan  2.  1.]
========== ARTRALGIA ==========
[ 2.  1. nan]
========== ARTRITE ==========
[ 2.  1. nan]
========== AUTO_IMUNE ==========
[ 2.  1. nan]
========== CEFALEIA ==========
[ 2.  1. nan]
========== CLASSI_FIN ==========
[ 5. 10.  8. 11. 12. nan]
========== CLINC_CHIK ==========
[nan  1.  2.]
========== COMPLICA ==========
[nan]
========== CONJUNTVIT ==========
[ 2.  1. nan]
========== CON_FHD ==========
[nan]
========== COPAISINF ==========
[ nan   1.  31.  44. 126.  68. 140.  32. 186.  21. 129.  57.  72. 156.
 111.]
========== COUFINF

TODO: Cruzar informações com dicionário de dados para checar se há colunas redundantes ou possíveis simplificações

### Verificando porcentagem de valores nulos

In [13]:
null_information = df.isnull().mean()

In [14]:
df_analysis = (null_information * 100).to_frame("perc_null")

# 2. Cria a nova coluna 'years_found' mapeando o índice (nome da coluna)
#    com o dicionário column_tracker.
#    Convertemos para string (str) para que a lista apareça inteira na célula da tabela.
df_analysis["years_found"] = df_analysis.index.map(lambda col: str(column_tracker.get(col, [])))

# 3. Gera o HTML ordenado pela porcentagem de nulos
HTML(df_analysis.sort_values("perc_null").to_html(max_rows=None, float_format="%.6f"))

,perc_null,years_found
DT_NOTIFIC,0.000000,"[20, 21, 22]"
TP_NOT,0.000000,"[20, 21, 22]"
SG_UF_NOT,0.000000,"[20, 21, 22]"
SG_UF,0.000000,"[20, 21, 22]"
NU_IDADE_N,0.000000,"[20, 21, 22]"
ID_MUNICIP,0.000000,"[20, 21, 22]"
NU_ANO,0.000000,"[20, 21, 22]"
SEM_PRI,0.000026,"[20, 21, 22]"
ID_PAIS,0.000308,"[20, 21, 22]"
CS_SEXO,0.000949,"[20, 21, 22]"


### Colunas removidas

|Coluna    |NULL PERCENT|NOME                               |SIGNIFICADO                                                                                        |OBS                                                                            |APAGADO|
|----------|------------|-----------------------------------|---------------------------------------------------------------------------------------------------|-------------------------------------------------------------------------------|-------|
|UF        |97.140192   |UF (Geral/Duplicado)               |Provável duplicata ou alias para SG_UF (UF de Residência). Ver campo SG_UF.                        |Ver definição de SG_UF no documento.                                           |       |
|DOENCA_TRA|100.000000  |Doença do Trabalho                 |Indica se a doença está ou não relacionada ao trabalho.                                            |Habilitado se confirmado.                                                      |       |
|MIGRADO_W |100.000000  |Migração Windows                   |Identifica se o registro é oriundo da rotina de migração da base Windows.                          |Campo interno.                                                                 |       |

In [15]:
df = df.drop(columns=["UF", "DOENCA_TRA", "MIGRADO_W"])

## Load (Carregamento) dos dados

In [ ]:
import pandas as pd
import sqlite3
from sqlalchemy import create_engine
import gc

db_path = 'dengue_dw.db'
engine = create_engine(f'sqlite:///{db_path}')

column_map = {
    # Notificação
    "NU_IDADE_N": "idade_paciente",
    "NU_ANO": "ano",
    "TP_NOT": "tipo",
    "SEM_PRI": "semana_inicio",
    "DT_SIN_PRI": "data_inicio_sintomas",
    "ID_MN_RESI": "municipio_residencia",
    "CS_GESTANT": "idade_gestacional",
    "CLASSI_FIN": "Classificacao_final",
    "DT_ENCERRA": "data_encerramentoo",
    "TP_SISTEMA": "Sistema",
    "DT_INVEST": "data_inicio_investigacao",
    "CRITERIO": "criterio_confirmacao",
    "EVOLUCAO": "evolucao_caso",
    "HOSPITALIZ": "hospitalizacao",
    "SEM_NOT": "semana_notificacao",
    "CS_FLXRET": "fluxo_retorno",
    "DT_SORO": "data_coleta_igm_dengue",
    "DT_NS1": "data_coleta_ns1",
    "TPAUTOCTO": "caso_autocne",
    "COPAISINF": "pais_infeccao",
    "COUFINF": "uf_infeccao",
    "COMUNINF": "municipio_infeccao",
    "DT_INTERNA": "data_internacao",
    "MUNICIPIO": "municipio_internacao",
    "DT_PCR": "data_coleta_rt_pcr",
    "DT_OBITO": "data_obito",
    "NDUPLIC_N": "duplicada",
    "FLXRECEBI": "receb_flux_retorno",
    "DT_DIGITA": "data_digitacao",

    # Unidade Saude
    "SG_UF_NOT": "UF_unidade",
    "ID_UNIDADE": "codigo_unidade",
    "ID_REGIONA": "id_regional_saude",

    # Paciente
    "SG_UF": "UF_residencia",
    "ID_MN_RESI": "municipio_residencia",
    "ID_PAIS": "pais_residencia_exterior",
    "CS_SEXO": "sexo",
    "CS_RACA": "raça",
    "ID_RG_RESI": "id_regional_saude_residencia",
    "CS_ESCOL_N": "Escolaridade",
    "ID_OCUPA_N": "Ocupação",

    # Quadro Clinico / Sintomas
    "FEBRE": "sc_FEBRE",
    "LEUCOPENIA": "sc_LEUCOPENIA",
    "EXANTEMA": "sc_EXANTEMA",
    "DOR_RETRO": "sc_DOR_RETRO",
    "HIPERTENSA": "dpe_hipertensao",
    "CONJUNTVIT": "sc_CONJUNTVIT",
    "CEFALEIA": "sc_CEFALEIA",
    "DOR_COSTAS": "sc_DOR_COSTAS",
    "DIABETES": "dpe_diabetes",
    "AUTO_IMUNE": "dpe_auto_imunes",
    "ACIDO_PEPT": "dpe_ácido_péptica",
    "ARTRALGIA": "sc_Artralgia_intensa",
    "ARTRITE": "sc_Artrite",
    "HEMATOLOG": "dpe_Doenças_hematológicas",
    "HEPATOPAT": "dpe_Hepatopatias",
    "LACO": "sc_prova_laço",
    "RENAL": "dpe_Doença_renal_crônica",
    "VOMITO": "sc_Vômito",
    "NAUSEA": "sc_Náusea",
    "MIALGIA": "sc_Mialgia",
    "PETEQUIA_N": "sc_Petéquias",
    "ALRM_PLAQ": "sa_plaquetas",
    "GRAV_HIPOT": "dg_hipotensão",
    "GRAV_TAQUI": "dg_taquicardia",
    "GRAV_PULSO": "dg_pulso",
    "GRAV_EXTRE": "dg_extremidades",
    "GRAV_INSUF": "dg_insuf_Resp",
    "GRAV_ENCH": "dg_enchimento",
    "GRAV_MELEN": "dg_melena",
    "GRAV_CONV": "dg_pa",
    "GRAV_CONSC": "dg_consciência",
    "GRAV_HEMAT": "dg_hematêmese",
    "GRAV_METRO": "dg_metrorragia",
    "GRAV_ORGAO": "dg_órgãos",
    "GRAV_SANG": "dg_sangramento",
    "GRAV_MIOC": "dg_miocardite",
    "GRAV_AST": "dg_ast_alt",
    "DT_GRAV": "data_dengue_grave",
    "GENGIVO": "mh_gengivorragia",
    "EPISTAXE": "mh_epistaxe",
    "CON_FHD": "caso_fhd_scd",
    "COMPLICA": "complicacoes",
    "EVIDENCIA": "evidencia_extravasamento",
    "METRO": "mh_metrorragia",
    "MANI_HEMOR": "manifestacoes_hemorragicas",
    "LACO_N": "prova_laço_especif",
    "HEMATURA": "mh_hematúria",
    "PETEQUIAS": "mh_petéquias",
    "PLASMATICO": "extravasamento_plasmatico",
    "PLAQ_MENOR": "plaquetas_menor",
    "SANGRAM": "mh_sangramento_gastrointestinal",
    "ALRM_ABDOM": "sa_dor_abdominal",
    "ALRM_HIPOT": "sa_hipotensão",
    "ALRM_SANG": "sa_sangramento",
    "ALRM_VOM": "sa_vômitos",
    "ALRM_LETAR": "sa_letargia",
    "ALRM_HEMAT": "sa_hematócrito",
    "ALRM_LIQ": "sa_líquidos",
    "ALRM_HEPAT": "sa_hepatomegalia",
    "DT_ALRM": "data_sa",
    "SOROTIPO": "sorotipo",
    "CLINC_CHIK": "tipo_chikungunya",

    # Exames
    "RESUL_SORO": "resultado_igm",
    "RESUL_NS1": "resultado_ns1",
    "RESUL_PCR_": "resultado_rt_pcr",
    "RESUL_VI_N": "resultado_isolamento_viral",
    "IMUNOH_N": "resultado_imunohistoquímica",
    "HISTOPA_N": "resultado_histopatologia",
    "RES_CHIKS1": "resultado_IgM_Chik_S1",
    "RES_CHIKS2": "resultado_IgM_Chik_S2",
    "RESUL_PRNT": "resultado_PRNT",
    "DT_VIRAL": "data_coleta_isolamento_viral",
    "DT_CHIK_S1": "data_coleta_igm_chik_S1",
    "DT_PRNT": "data_coleta_prnt",
    "DT_CHIK_S2": "data_coleta_igm_chik_S2",

    # Chaves e Tempo
    'DT_NOTIFIC': 'data_notificacao',
    'ID_MUNICIP': 'id_municipio',
    'NU_ANO': 'ano',
}

new_names = [column_map.get(col, col) for col in df.columns]
df.columns = new_names

# Tratamento de Data (Essencial para Dim_Tempo)
df['data_notificacao'] = pd.to_datetime(df['data_notificacao'], errors='coerce')

# Criar um ID único para cada notificação (Chave Primária da Fato)
df['id_notificacao'] = df.index + 1

print("Dados preliminares preparados.")

Dados preliminares preparados.


In [18]:
dim_tempo = pd.DataFrame(df['data_notificacao'].unique(), columns=['data'])
dim_tempo = dim_tempo.dropna()
dim_tempo['ano'] = dim_tempo['data'].dt.year
dim_tempo['mes'] = dim_tempo['data'].dt.month
dim_tempo['dia'] = dim_tempo['data'].dt.day
dim_tempo['semana_ano'] = dim_tempo['data'].dt.isocalendar().week
dim_tempo['dia_semana'] = dim_tempo['data'].dt.day_name()

# Load no Banco
dim_tempo.to_sql('dim_tempo', engine, if_exists='replace', index=False, chunksize=1000)
print("Dimensão Tempo carregada.")

Dimensão Tempo carregada.


In [19]:

# Load Dimensão Localidade
dim_localidade = df[['id_municipio']].drop_duplicates().dropna()
dim_localidade.to_sql('dim_localidade', engine, if_exists='replace', index=False, chunksize=1000)
print("Dimensão Localidade carregada.")

Dimensão Localidade carregada.


In [20]:
# Dimensão Paciente (Demografia)
cols_paciente = ['id_notificacao', 'UF_residencia', 'municipio_residencia', 'pais_residencia_exterior', 'sexo', 'raça', 'id_regional_saude', 'Escolaridade', 'Ocupação']
dim_paciente = df[cols_paciente]
dim_paciente.to_sql('dim_paciente', engine, if_exists='replace', index=False, chunksize=1000)
print("Dimensão Paciente carregada.")

Dimensão Paciente carregada.


In [21]:
# Dimensão Clínica (Sintomas)
cols_clinica = ['id_notificacao', "sc_FEBRE", "sc_LEUCOPENIA", "sc_EXANTEMA", "sc_DOR_RETRO", "dpe_hipertensao", "sc_CONJUNTVIT", "sc_CEFALEIA", "sc_DOR_COSTAS", "dpe_diabetes", "dpe_auto_imunes", "dpe_ácido_péptica", "sc_Artralgia_intensa", "sc_Artrite", "dpe_Doenças_hematológicas", "dpe_Hepatopatias", "sc_prova_laço", "dpe_Doença_renal_crônica", "sc_Vômito", "sc_Náusea", "sc_Mialgia", "sc_Petéquias", "sa_plaquetas", "dg_hipotensão", "dg_taquicardia", "dg_pulso", "dg_extremidades", "dg_insuf_Resp", "dg_enchimento", "dg_melena", "dg_pa", "dg_consciência", "dg_hematêmese", "dg_metrorragia", "dg_órgãos", "dg_sangramento", "dg_miocardite", "dg_ast_alt", "data_dengue_grave", "mh_gengivorragia", "mh_epistaxe", "caso_fhd_scd", "complicacoes", "evidencia_extravasamento", "mh_metrorragia", "manifestacoes_hemorragicas", "prova_laço_especif", "mh_hematúria", "mh_petéquias", "extravasamento_plasmatico", "plaquetas_menor", "mh_sangramento_gastrointestinal", "sa_dor_abdominal", "sa_hipotensão",  "sa_sangramento",  "sa_vômitos",  "sa_letargia",  "sa_hematócrito",  "sa_líquidos",  "sa_hepatomegalia",  "data_sa",  "sorotipo",  "tipo_chikungunya", ]
dim_clinica = df[cols_clinica]
dim_clinica.to_sql('dim_clinica', engine, if_exists='replace', index=False, chunksize=1000)
print("Dimensão Clínica carregada.")

Dimensão Clínica carregada.


In [22]:
# Dimensão Exames
cols_exames = ['id_notificacao', "resultado_igm", "resultado_ns1", "resultado_rt_pcr", "resultado_isolamento_viral", "resultado_imunohistoquímica", "resultado_histopatologia", "resultado_IgM_Chik_S1", "resultado_IgM_Chik_S2", "resultado_PRNT", "data_coleta_isolamento_viral", "data_coleta_igm_chik_S1", "data_coleta_prnt", "data_coleta_igm_chik_S2"]
dim_exames = df[cols_exames]
dim_exames.to_sql('dim_exames', engine, if_exists='replace', index=False, chunksize=1000)
print("Dimensão Exames carregada.")

Dimensão Exames carregada.


In [23]:
# Dimensão Unidade de Saúde
cols_unidade_saude = ["id_municipio", "UF_unidade", "codigo_unidade", "id_regional_saude"]
dim_unidade_saude = df[cols_unidade_saude]
dim_unidade_saude.to_sql('dim_unidade_saude', engine, if_exists='replace', index=False, chunksize=1000)
print("Dimensão Exames carregada.")

Dimensão Exames carregada.


In [25]:
cols_fato = [
    'id_notificacao',      # PK
    'data_notificacao',    # FK para Dim_Tempo (Join pela data)
    'id_municipio',        # FK para Dim_Localidade
    'codigo_unidade',      # FK para Dim_Unidade_Saude

     "idade_paciente",
     "ano",
     "tipo",
     "semana_inicio",
     "data_inicio_sintomas",
     "municipio_residencia",
     "idade_gestacional",
     "Classificacao_final",
     "data_encerramentoo",
     "Sistema",
     "data_inicio_investigacao",
     "criterio_confirmacao",
     "evolucao_caso",
     "hospitalizacao",
     "semana_notificacao",
     "fluxo_retorno",
     "data_coleta_igm_dengue",
     "data_coleta_ns1",
     "caso_autocne",
     "pais_infeccao",
     "uf_infeccao",
     "municipio_infeccao",
     "data_internacao",
     "municipio_internacao",
     "data_coleta_rt_pcr",
     "data_obito",
     "duplicada",
     "receb_flux_retorno",
     "data_digitacao"
  ]

fato_notificacao = df[cols_fato]

fato_notificacao.to_sql('fato_notificacao', engine, if_exists='replace', index=False, chunksize=1000)

print("Tabela Fato carregada com sucesso!")

Tabela Fato carregada com sucesso!


In [26]:
# Teste: Contagem de casos por Data e Sexo
query = """
SELECT
    f.data_notificacao,
    p.sexo,
    COUNT(*) as total_casos
FROM fato_notificacao f
JOIN dim_paciente p ON f.id_notificacao = p.id_notificacao
GROUP BY f.data_notificacao, p.sexo
ORDER BY total_casos DESC
LIMIT 10;
"""

df_resultado = pd.read_sql(query, engine)
print(df_resultado)

             data_notificacao sexo  total_casos
0  2022-04-25 00:00:00.000000    F        12282
1  2022-04-18 00:00:00.000000    F        12050
2  2022-05-09 00:00:00.000000    F        11650
3  2022-05-02 00:00:00.000000    F        10795
4  2022-04-25 00:00:00.000000    M        10532
5  2022-04-19 00:00:00.000000    F        10330
6  2022-04-26 00:00:00.000000    F        10327
7  2022-05-10 00:00:00.000000    F         9979
8  2022-04-18 00:00:00.000000    M         9971
9  2022-05-03 00:00:00.000000    F         9905
